<a href="https://colab.research.google.com/github/RileyWClarke/MLTSA_RClarke/blob/master/HW9/mltsa_hw9_rclarke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from matplotlib import pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
import pandas as pd
% matplotlib inline

In [0]:
plt.style.use("https://raw.githubusercontent.com/fedhere/MLTSA_FBianco/master/fbb.mplstyle")

In [4]:
!pip install kaggle

In [6]:
from google.colab import drive

drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
cd gdrive/My\ Drive/

/content/gdrive/My Drive


In [8]:
ls kaggle.json

kaggle.json


In [0]:
!chmod 600 kaggle.json

In [0]:
import json
import os
envs = json.load(open("kaggle.json", "r"))
os.environ["KAGGLE_USERNAME"] = envs['username']
os.environ["KAGGLE_KEY"] = envs['key']

In [11]:
!kaggle competitions list -s PLAsTiCC

ref            deadline             category   reward  teamCount  userHasEntered  
-------------  -------------------  --------  -------  ---------  --------------  
PLAsTiCC-2018  2018-12-17 23:59:00  Featured  $25,000       1094            True  


In [12]:
cd plasticc

/content/gdrive/My Drive/plasticc


In [13]:
cd data

/content/gdrive/My Drive/plasticc/data


In [14]:
#!kaggle competitions download PLAsTiCC-2018

test_set_batch7.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_set_batch11.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
training_set.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_set_batch5.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
data_note.pdf.zip: Skipping, found more recently modified local copy (use --force to force download)
test_set_batch3.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_set_batch2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
training_set_metadata.csv: Skipping, found more recently modified local copy (use --force to force download)
test_set.csv.zip: Skipping, found more re

In [15]:
ls

allfeatures                test_set_batch2.csv.zip  test_set.csv.zip
allfeatures.npy            test_set_batch3.csv.zip  test_set_metadata.csv.zip
data_note.pdf.zip          test_set_batch4.csv.zip  test_set_sample.csv.zip
metadata                   test_set_batch5.csv.zip  training_set.csv
sample_submission.csv.zip  test_set_batch6.csv.zip  training_set.csv.zip
test_set_batch10.csv.zip   test_set_batch7.csv.zip  training_set_metadata.csv
test_set_batch11.csv.zip   test_set_batch8.csv.zip
test_set_batch1.csv.zip    test_set_batch9.csv.zip


In [16]:
!unzip training_set.csv.zip

Archive:  training_set.csv.zip
replace training_set.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: training_set.csv        


In [17]:
lcvs = pd.read_csv('training_set.csv')
lcvs.head()

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [66]:
starts = lcvs.groupby(["object_id"])[["mjd"]].min()
starts

,mjd
object_id,
615,59750.4229
713,59825.2600
730,59798.3205
745,59770.3662
1124,59750.4229
...,...
130739978,59790.3778
130755807,59581.3605
130762946,59620.2130


In [104]:
starts['mjd'].loc[starts.index == 615].values[0]

59750.4229

In [105]:
lcvs['object_id'].values[0]

615

Create new domain where each object is time indexed with its first observation being zero

In [108]:
#Very stupid and expensive way to do this:

mjd_new = np.zeros(lcvs.shape[0])

for i, obj in enumerate(lcvs['object_id'].values):

  if lcvs['mjd'][i] == starts['mjd'].loc[starts.index == obj].values[0]:
    pass

  else:
    mjd_new[i] = lcvs['mjd'][i] - starts['mjd'].loc[starts.index == obj]

 
mjd_new

array([0.000000e+00, 7.700000e-03, 1.540000e-02, ..., 8.606095e+02,
       8.755844e+02, 8.776242e+02])

In [109]:
lcvs['mjd_new'] = mjd_new
lcvs.head()

,object_id,mjd,passband,flux,flux_err,detected,mjd_new
0,615,59750.4229,2,-544.810303,3.622952,1,0.0000
1,615,59750.4306,1,-816.434326,5.553370,1,0.0077
2,615,59750.4383,3,-471.385529,3.801213,1,0.0154
3,615,59750.4450,4,-388.984985,11.395031,1,0.0221
4,615,59752.4070,2,-681.858887,4.041204,1,1.9841


In [0]:
lcvs.to_pickle('lcvs.pkl')

In [119]:
lcvs_p1 = lcvs.loc[lcvs['passband'].values == 1]
lcvs_p1.head()

,object_id,mjd,passband,flux,flux_err,detected,mjd_new
1,615,59750.4306,1,-816.434326,5.553370,1,0.0077
5,615,59752.4147,1,-1061.457031,6.472994,1,1.9918
10,615,59767.3045,1,-815.188599,5.293019,1,16.8816
15,615,59770.2256,1,-820.042786,5.875329,1,19.8027
20,615,59779.3265,1,-921.002502,6.306800,1,28.9036


In [125]:
lcvs_p1.tail()

,object_id,mjd,passband,flux,flux_err,detected,mjd_new
1421652,130779836,60177.0336,1,54.425945,19.356459,0,466.6206
1421668,130779836,60434.3008,1,-34.273514,19.637548,0,723.8878
1421678,130779836,60491.1907,1,-13.734149,19.639900,0,780.7777
1421692,130779836,60530.0978,1,2.986908,19.109673,0,819.6848
1421701,130779836,60560.0459,1,14.894439,18.947685,0,849.6329


Flatten time series

In [0]:
lcvsp1_flat = np.concatenate((lcvs_p1['mjd_new'].values, lcvs_p1['flux'].values))#This vector gets fed to autoencoder

In [124]:
lcvsp1_flat

array([ 7.7000000e-03,  1.9918000e+00,  1.6881600e+01, ...,
       -1.3734149e+01,  2.9869080e+00,  1.4894439e+01])